# Numpy, Pandas 数据处理之小技巧


Edward.Fu · 1 天前

https://zhuanlan.zhihu.com/p/21337237

今日是端午4天假期的最后一天，自己的2台电脑一台在处理数据，另一台电源适配器也坏掉了。感觉闲来无事，在此就简单聊聊numpy和pandas的各种冷知识、小技巧和疑难杂症。
个人觉得关于numpy和pandas的坑太大，不好填，在此就不系统的介绍了，各位可以买些这块的资料先自己看看。

这里就着重聊聊一些使用过程中常用到但教科书里找不着的问题，省的各位朋友还跑去stackoverflow找答案。

### 问题一、dataframe里面.values，.iloc，.ix，.loc的区别？

只有values是将原本dataframe数据强制转化为numpy格式的数据来索引，其他3个都是对dataframe本身的数据来索引，其中iloc是对基于values的位置来索引调用，loc是对index和columns的位置来索引，而ix则是先用loc的方式来索引，索引失败就转成iloc的方式；

### 问题二、可否有两层，或2层以上的columns或index？有的话如何索引？

可以，索引的话如果用loc或ix，则默认是用第一层的index或columns，最简单的方式是类似这样：

```python
example.loc[index1, columns1].loc[index2, columns2]
```

### 问题三、list, dict, numpy.ndarray, dataframe数据格式如何转换？

```python
# 1. list转化为numpy.ndarray：
np.array(example)

# 2. numpy.ndarray转化为list：
list(example)

# 3. dict转化为dataframe:
example['a'] = {'bb': 2, 'cc': 3}
eee = pd.DataFrame(example)

# 4. numpy.ndarray转化为dataframe:
pd.DataFrame(example)

# 5. dataframe转化为numpy.ndarray：
example.values[:, :]
```

### 问题四、numpy.ndarray和dataframe如何填补nan，inf？

```python
# 1. 对于numpy.ndarray来说：
example = np.where(np.isnan(example), 0, example)
example = np.where(np.isnan(example), 0, example)

# 2. 对于dataframe来说：
example.fillna()
# 还可以用
example.replace(a, b)
```

### 问题五、各种IO的效率快慢问题？

1. np读写效率最高，但最费硬盘空间，比如np.load(), np.save();

2. csv其次，比如pd.Dataframe.to_csv()，pd.load_csv()；

3. txt读写，当然也可以很快，但是需要频繁的split，对格式规范的数据比较麻烦；

4. 至于简单的excel和word，可以用xlrd,xlwt来操作；

### 问题六、关于常见的os操作，包括新建文件夹、遍历文件夹的操作问题？

```python
# 1. 新建文件夹：
if not os.path.isdir(path_out):
    os.makedirs(path_out)
    
# 2. 遍历所有文件和子文件夹：
for a, b, filenames in os.walk(path_data):
    for filename in filenames:
        ...
    
# 只遍历当前文件，不包含子文件夹：
for a, b, filenames in os.walk(path_data):
    for filename in filenames:
        if a == path_data:
            ...
```

### 问题七、numpy.ndarray和dataframe如何选取满足条件的行和列数据？

```python
# 1. 根据新的columns来选取：
frame_[newcolumns]

# 2. 根据新的index来选取：
frame_[frame_.index.isin(newindex)]

# 3. 根据某一行或者列的条件来选取：
# 假如是根据dataframe的第一列，必须大于start_time这个常数，
frame_ = frame_.ix[:, frame_.ix[0, :] >= start_date]
# 或者是根据dataframe的第一行，必须大于start_time这个常数，
frame_ = frame_.ix[frame_.ix[:, 0] >= start_date, :]
```

### 问题八、如何计算相关性矩阵？

将y和所有x放入到`sample = numpy.ndarray`下，然后直接`np.corrcoef(sample)`，默认的是皮尔森相关系数，当然，也可以用ranked correlation，也就是spearman correlation，可以直接用`scipy.stats.spearmanr`。

### 问题九、如何取出一串字符串里面的字母或者数字？

```python
# 1. 取出example里面的数字：
int(''.join(x for x in example if x.isdigit()))

# 2. 取出example里面的字母：
(''.join(x for x in example if x.alpha()))
```

### 问题十、各种merge操作？

```python
# 1. 纵向merge格式为numpy.ndarray的数据：
np.hstack((example1, example2))

# 2. 纵向merge格式为dataframe的数据，
#    并根据dataframe的index来merge，merge后保留原本各自列的所有index，
#    其他没有该index的列则对应数值为nan：
pd.concat([example1, example2], axis=1)

# 3. 纵向merge，但是只保留公共的index行：
example.sort_index(axis=1, inplace=True)

# 4. 横向merge格式为numpy.ndarray的数据：
np.vstack((example1, example2))

# 5. 横向merge格式为dataframe的数据，
#    并根据dataframe的column来merge，merge后保留原本的index和columns，
#    其他没有该index或columns的列则对应数值为np.nan：
pd.concat([example1, example2], axis=0)

# 6. 横向merge，但是只保留公共的columns列：
example.sort_index(axis=0, inplace=True)
```

### 问题十一、对dataframe数据的index统一加一个后缀

比如对原本dataframe下的`index=['aa', 'cc', 'dddddd']`的，统一加上一个`_5m`的后缀，通常的操作大家一般就是直接`example.index = [x + '_5m' for x in example.index]`，这个其实会产生些小问题，因为默认的`index`是`pandas.indexes.base.Index`，这个格式可能会默认index里面数据的长度是确定的，导致加`_5m`后缀失败，所以需要先把格式强制转化为list, 像这样：`example.index = [x + '_5m' for x in list(example.index)]`

In [ ]:
先就这些吧，其他的想到再慢慢加。